In [1]:
from reconcilation import *

## Reading input files

In [2]:
PRODUCT = pd.read_csv('data/DPS_PRODUCT.csv')
LOCATION = pd.read_csv('data/DPS_LOCATION.csv')
CUSTOMER = pd.read_csv('data/DPS_CUSTOMER.csv')
DISTR_CHANNEL = pd.read_csv('data/DPS_DISTR_CHANNEL.csv')

PRODUCT.columns = PRODUCT.columns.str.lower()
LOCATION.columns = LOCATION.columns.str.lower()
CUSTOMER.columns = CUSTOMER.columns.str.lower()
DISTR_CHANNEL.columns = DISTR_CHANNEL.columns.str.lower()

hierarchies = {
    'product' : PRODUCT,
    'location' : LOCATION,
    'customer' : CUSTOMER,
    'distr_channel' : DISTR_CHANNEL
}

## Config example

In [3]:
config = {
    'tgt_type' : 'POS',
    'value_src' : 'SALES_QTY',
    'act_flag' : 1,
    'vf_product_lvl' : 1,
    'vf_location_lvl' : 1,
    'vf_customer_lvl' : 1,
    'vf_distr_channel_lvl' : 1,
    'vf_time_lvl' : 'MONTH',
    'ml_product_lvl' : 7,
    'ml_location_lvl' : 5,
    'ml_customer_lvl' : 5,
    'ml_distr_channel_lvl' : 1,
    'ml_time_lvl' : 'WEEK'
}

IB_HIST_END_DT = datetime.datetime(2022, 6, 30)
IB_FC_HORIZ = 30
delays_config_length = 30

## Generating data

In [4]:
VF_FORECAST, ML_FORECAST, VF_TS_SEGMENTS = generate_data(config, IB_HIST_END_DT, hierarchies)

In [5]:
VF_FORECAST.head()

,PERIOD_DT,product_lvl_id1,customer_lvl_id1,location_lvl_id1,distr_channel_lvl_id1,FORECAST_VALUE
0,2022-07-01,10084,1000000,100000,2,123.799232
1,2022-08-01,10084,1000000,100000,2,448.831935
2,2022-09-01,10084,1000000,100000,2,791.160317
3,2022-10-01,10084,1000000,100000,2,320.614439
4,2022-11-01,10084,1000000,100000,2,208.429293


In [6]:
ML_FORECAST.head()

,PERIOD_DT,product_lvl_id7,customer_lvl_id5,location_lvl_id5,distr_channel_lvl_id1,FORECAST_VALUE,DEMAND_TYPE,ASSORTMENT_TYPE
0,2022-07-04,70001,5000001,500015,2,897.452476,promo,old
1,2022-07-04,70001,5000001,500015,2,869.544564,regular,old
2,2022-07-04,70001,5000001,500014,2,1164.669228,promo,old
3,2022-07-04,70001,5000001,500014,2,804.564863,regular,new
4,2022-07-04,70001,5000001,500015,2,750.825399,promo,new


## Running the algorighm

In [7]:
mid_reconciled_forecast, reconciled_forecast = reconcilation_algorithm(
    VF_FORECAST, ML_FORECAST, VF_TS_SEGMENTS, config, IB_HIST_END_DT,
    IB_FC_HORIZ, delays_config_length, hierarchies
)

In [8]:
mid_reconciled_forecast

,PERIOD_END_DT,product_lvl_id1,customer_lvl_id1,location_lvl_id1,distr_channel_lvl_id1,VF_FORECAST_VALUE_REC,ML_FORECAST_VALUE,DEMAND_TYPE,ASSORTMENT_TYPE
1,2022-08-01,10084,1000000,100000,2,448.831935,None,Regular,old
2,2022-09-01,10084,1000000,100000,2,791.160317,None,Regular,old
3,2022-10-01,10084,1000000,100000,2,320.614439,None,Regular,old
4,2022-11-01,10084,1000000,100000,2,208.429293,None,Regular,old
5,2022-12-01,10084,1000000,100000,2,661.933788,None,Regular,old
6,2023-01-01,10084,1000000,100000,2,413.936725,None,Regular,old
7,2023-02-01,10084,1000000,100000,2,649.882093,None,Regular,old
8,2023-03-01,10084,1000000,100000,2,499.809779,None,Regular,old
9,2023-04-01,10084,1000000,100000,2,623.984896,None,Regular,old
10,2023-05-01,10084,1000000,100000,2,645.580974,None,Regular,old


In [9]:
reconciled_forecast

,product_lvl_id7,customer_lvl_id5,location_lvl_id5,distr_channel_lvl_id1,ML_FORECAST_VALUE,DEMAND_TYPE,ASSORTMENT_TYPE,product_lvl_id1,customer_lvl_id1,location_lvl_id1,PERIOD_DT,PERIOD_END_DT,VF_FORECAST_VALUE_REC,SEGMENT_NAME
0,70001,5000001,500015,2,897.452476,promo,old,10084,1000000,100000,2022-07-04,2022-07-10,42.293673,Name of segment 0
1,70001,5000001,500015,2,869.544564,regular,old,10084,1000000,100000,2022-07-04,2022-07-10,40.978474,Name of segment 0
2,70001,5000001,500015,2,750.825399,promo,new,10084,1000000,100000,2022-07-04,2022-07-10,35.383672,Name of segment 0
3,70001,5000001,500015,2,5.453898,regular,old,10084,1000000,100000,2022-07-04,2022-07-10,0.257022,Name of segment 0
4,70001,5000001,500015,2,873.318923,regular,old,10084,1000000,100000,2022-07-04,2022-07-10,41.156346,Name of segment 0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73035,70083,5000003,500017,2,169.650185,promo,old,10084,1000000,100000,2022-07-25,2022-07-31,7.818198,Name of segment 0
73036,70083,5000003,500017,2,90.830228,promo,new,10084,1000000,100000,2022-07-25,2022-07-31,4.185841,Name of segment 0
73037,70083,5000003,500017,2,767.048999,regular,old,10084,1000000,100000,2022-07-25,2022-07-31,35.348864,Name of segment 0
73038,70083,5000003,500017,2,746.047323,regular,new,10084,1000000,100000,2022-07-25,2022-07-31,34.381018,Name of segment 0
